# REINFORCE
---
In this notebook you are going to implement the `REINFORCE` algorithm.

<a target="_blank" href="https://colab.research.google.com/github/PrzemekSekula/ReinforcementLearningClasses/blob/master/REINFORCE/REINFORCE.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>

#### Pseudocode (general case):

1. Use the policy $\pi_w$ to collect $m$ trajectories $\tau^{(1)}, \tau^{(2)}, ..., \tau^{(m)}$ with horizon $H$. We refer to the $i$-th trajectory as:
$$\tau^{(i)} = (s_0^{(i)}, a_0^{(i)}, r_1^{(i)}, s_1^{(i)}, a_1^{(i)}, r_2^{(i)}, ...., s_H^{(i)}, a_H^{(i)}, r_H^{(i)} s_{H+1}^{(i)})$$

2. Use the trajectories to estimate the gradient $\nabla_wU(w)$:
$$\nabla_wU(w) \approx \hat{g} := \frac{1}{m} \sum_{i=1}^m \sum_{t=0}^H \nabla \log{\pi(a_t^{(i)} | s_t^{(i)})} * R(\tau^{(i)}) $$ 

where $R(\tau^{(i)})=\sum_{t=1}^H \gamma^{t-1}*r_t^{(i)}$ is a sum of all (discounted) rewards collected during the trajectory $\tau^{(i)}$.

3. Update the weights of the policy:
    $$w \leftarrow w + \alpha * \hat{g}$$

4. If the results are not satisfactory, go to step 1.

#### Pseudocode (update policy with each trajectory)
In our implementation we are going to simplify two aspects of this algorithm:
- We are collecting only one trajectory and update the policy with it.
- We assume that the trajectory always ends at terminal state (or after `max_t` steps if terminal state was not reached).

With these asumptions the `REINFORCE` pseudocode looks as follows:

1. Use the policy $\pi_w$ to collect a trajectory:
$$\tau = (s_0, a_0, r_1, s_1, a_1, r_2, ...., s_{T-1}, a_{T-1}, r_T, s_{T})$$

2. Use the collected trajectory to estimate the gradient $\nabla_wU(w)$:
$$\nabla_wU(w) \approx \hat{g} := \sum_{t=0}^H \nabla \log{\pi(a_t | s_t)} * R(\tau) $$ 

where $R(\tau)=\sum_{t=1}^T \gamma^{t-1}*r_t$ is a sum of all (discounted) rewards collected during the trajectory $\tau$.

3. Update the weights of the policy:
    $$w \leftarrow w + \alpha * \hat{g}$$

4. If the results are not satisfactory, go to step 1.    

#### Pseudocode - implementation details
It is possible to simplify things even more during implementation. Please note, that while we are collecting a trajectory, we are using a policy (neural network) to sample the action. This requires computing the probabilities of each action. Instead of collecting the trajectory and then reusing this network to compute this probability again, we may simply collect the probability (or even the log probability) and the corresponding rewards. With this assumption, the first step of the algorithm looks as follows:

1. Using policy $\pi_w$ generate the episode. Collect the log probabilities $\log{\pi(a_t | s_t)}$ and rewards $r_t$. At the end of the episode you should have two list
$$LogProbabilities = (\log{\pi(a_0 | s_0)}, \log{\pi(a_1 | s_1)}, ..., \log{\pi(a_{T-1} | s_{T-1})})$$
$$Rewards = (r_1, r_2, ..., r_T)$$

Steps 2, 3, and 4 are the same, yet as we collected all the log probabilities, we do not have to use the policy to calculate them again.

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install --upgrade gym


In [ ]:
import os
import gym
import numpy as np
from collections import deque

import matplotlib.pyplot as plt
from IPython import display

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

%matplotlib inline

In [ ]:
def ipython_show_video(path: str) -> None:
    from IPython.display import HTML, display
    """Show a video at `path` within IPython Notebook."""
    if not os.path.isfile(path):
        raise NameError("Cannot access: {}".format(path))

    video = io.open(path, "r+b").read()
    encoded = base64.b64encode(video)

    display(HTML(
        data="""
        <video width="320" height="240" alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
        </video>
        """.format(encoded.decode("ascii"))
    ))


def show_latest_video(video_folder: str) -> str:
    """Show the most recently recorded video from video folder."""
    list_of_files = glob.glob(os.path.join(video_folder, "*.mp4"))
    latest_file = max(list_of_files, key=os.path.getctime)
    ipython_show_video(latest_file)
    return latest_file

In [ ]:
env = gym.make('CartPole-v1')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Task 1 - Policy
Create a policy (neural network) that you are going to train. Fill in the `__init()`, `forward` and `act` methods. 
- For the network use the parameters as follows:
    - Input layer: 4 neurons (size of the state vector)
    - Hidden layer: 16 neurons, relu activation function
    - Output layer: 2 neurons (number of actions), `softmax` activation function
- In `act` method:
    - compute probabilities (use `forward()`) 
    - sample action proportionally to probabilities (use `m.sample()`)
    - return: 
        - action (use `action.item()` to change tensor to a value) 
        - logarithm of probability of the selected action (use `m.log_prob(action`)


In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size=4, h_size=16, a_size=2):
        super(Policy, self).__init__()
        # Create neural network layers here
        # ENTER YOUR CODE HERE
        pass

    def forward(self, x):
        # Implement forward pass
        # ENTER YOUR CODE HERE
        
        return None
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = None # ENTER YOUR CODE HERE (computer probabilities of each action)
        m = Categorical(probs)
        action = None # ENTER YOUR CODE HERE (sample an action from the distribution)
        return None, None # ENTER YOUR CODE HERE (return action and log_prob(action))


policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

### Task 2 - Generate episode.
Your goal is to generate the episode and store both logarithmic probabilities of each selected action and corresponding rewards. You are going to need these probabilities and rewards to implement the `REINFORCE` algorithm. Fill in the `generate_episode` function. You should:
- determine the initial state by reseting the environment
- use `policy` to get both the next action and the probabilities of all actions
- perform the selected action

In [ ]:
def generate_episode(policy, max_t=1000):
    saved_log_probs = []
    rewards = []
    
    state, _ = None # ENTER YOUR CODE HERE (reset the environment and get the initial state)
    for t in range(max_t):
        action, log_prob = None # ENTER YOUR CODE HERE (select an action)
        state, reward, done, _, _ = None # ENTER YOUR CODE HERE (perform a step in the environment)
        
        saved_log_probs.append(log_prob)
        rewards.append(reward)
        
        if done:
            break
        
    return saved_log_probs, rewards

### Task 3: Update policy
- Use `generate_episode()` to obtain the logarithms of probabilities of selected actions and the rewards that correspond to these actions.
- Discount the rewards. This is not necessary if you use `gamma=1.0` yet better to have it in your code.
- Calculate the sum of discounted rewards $R(\tau)$.
- For each action calculate a corresponding loss as $loss_{elem} = -log{\pi(a|s)} * R(\tau)$ where:
    - $log{\pi(a|s)}$ is the logarithm of the probability of the selected action (you already have it)
    - $R(\tau)$ is a sum of all the discounted rewards (you should calculate it in the previous point)

*Note: Remember to use the **negative** log probability $-log{\pi(a|s)}$. `optimizer.step()` uses **Gradient Descent** algorithm and you want to use **Gradient Ascent** to maximize the expected rewards. Thus, you should use negative log probabilities to get minus gradients with `policy_loss.backwards()`.*

In [ ]:
def reinforce(n_episodes=2000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        
        saved_log_probs, rewards = None # ENTER YOUR CODE HERE (generate an episode)
        
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))        
        
        R_discounted = None # ENTER YOUR CODE HERE (compute discounted rewards)
 
        R = None # ENTER YOUR CODE HERE (compute cumulative rewards)
        
        policy_loss = []
        for log_prob in saved_log_probs:
            policy_loss.append(None) # ENTER YOUR CODE HERE (compute policy loss)
            
        policy_loss = torch.cat(policy_loss).sum()
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print(f'Episode {i_episode}\tAverage Score: {np.mean(scores_deque):.2f}')
        if np.mean(scores_deque)>=195.0:
            print(f'Environment solved in {i_episode} episodes!\tAverage Score: {np.mean(scores_deque):.2f}')
            break
        
    return scores
    
scores = reinforce()

In [ ]:
def plot_scores(scores):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()
plot_scores(scores)

Let's see how our policy behaves (it does not work in Windows)

In [ ]:
def visualize_actions(policy):
    env = gym.make('CartPole-v1', render_mode = 'rgb_array')

    state, _ = env.reset()
    img_list = [env.render()]

    for t in range(1000):
        action, _ = policy.act(state)
        state, reward, done, _, _ = env.step(action)
        img_list.append(env.render())
        state, reward, done, _, _ = env.step(action)
        if done:
            break 
    env.close()

    plt.ion()
    animation = plt.imshow(img_list[0])
    for img in img_list:
        animation.set_data(img) 
        plt.axis('off')
        display.display(plt.gcf())
        display.clear_output(wait=True)



    print (f'Visualization done. Episode ended after {t} steps.')

visualize_actions(policy)

### Credit Assignment Demo (Optional)

Modify your algorithm to use only the future rewards. Calculate $\hat{g}$ as:

$$\hat{g} := \sum_{t=0}^T R_t^{future} \nabla \log{\pi(a_t | s_t)} $$

where:
$$R_t^{future}=\sum_{i=t}^Tr_i$$

